In [1]:
# Colab cell
try:
    import google.colab
    IN_COLAB = True
    !wget -q https://raw.githubusercontent.com/Competencia-de-Climate-Change/WeatherBench/master/src/ml_training.py
    !pip install -q -U zarr
    !pip install -q -U fsspec
    !pip install -q -U gcsfs
    !pip install -q -U xarray
except:
    IN_COLAB = False

In [2]:
import xarray as xr
import fsspec

ds = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamanño en GB:", ds.nbytes / 1e9)

Tamanño en GB: 19.324476644


In [3]:
ds

<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

# Tree based methods for $Z500$, $T850$ and $T2M$

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pickle

from src.ml_training import *

# from dask_ml.wrappers import ParallelPostFit

def to_pickle(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
def read_pickle(fn):
    with open(fn, 'rb') as f:
        return pickle.load(f)
    
results = 'results'
import os
if not os.path.exists(results):
    os.makedirs(results)

/opt/tljh/user/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## Data Loading

In [5]:
z_500 = ds.z.sel(level=500).drop('level')
t_850 = ds.t.sel(level=850).drop('level')
t2m = ds.t2m

data = xr.merge([z_500, t_850, t2m])

data_train = data.sel(time=slice('2010', '2016'))
data_test = data.sel(time=slice('2017', '2018'))

z_test = load_test_data('z', ds=ds)
t_test = load_test_data('t', ds=ds)
t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

_, nlat, nlon = data_train.z.shape; nlat, nlon

Selecting from ds...
Selecting from ds...


(12, 14)

In [6]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test = (data_test - data_mean) / data_std

### Define combinations of training data

In [7]:
# Inputs and Outputs
experiments = [
    [['z'], ['z']],
    [['t'], ['t']],
    [['z', 't'], ['z', 't']],
    [['t2m'], ['t2m']],
    [['z', 't', 't2m'], ['z']],
    [['z', 't', 't2m'], ['t']],
    [['z', 't', 't2m'], ['t2m']],
]

lead_time = 3*24 # Predict 3 days from current time
lead_time_str = '3d'

verbose = False

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## Run Experiments

In [9]:
preds  = {
    'dt_reg' : [],
    'rf_reg' : [],
}

models = {
    'dt_reg' : [],
    'rf_reg' : [],
}

train_mse = {
    'dt_reg' : [],
    'rf_reg' : [],
}

test_mse = {
    'dt_reg' : [],
    'rf_reg' : [],
}

weighted_mse = {
    'dt_reg' : [],
    'rf_reg' : [],
}

model_args = [
    [DecisionTreeRegressor, 'dt_reg', {'max_depth': 5}], 
    [RandomForestRegressor, 'rf_reg', {'n_jobs': -1, 'verbose': 3}], 
]

for model in model_args:
    model_fun, model_str, extra_args = model
    
    print('Running:', model_str)
    for iter, (i, o) in enumerate(experiments):
        print(f'{iter}: Input variables = {i}; output variables = {o}')
        
        predictions, model_res, mse_train, mse_test = train_regression(model=model_fun,
                                                                       data=[data_train, data_test, nlat, nlon, data_std, data_mean],
                                                                       lead_time_h=lead_time, 
                                                                       input_vars=i, 
                                                                       output_vars=o,
                                                                       data_subsample=5,
                                                                       extra_args=extra_args,
                                                                       verbose=verbose)

        preds[model_str].append(predictions)
        models[model_str].append(model_res)

        # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
        r = compute_weighted_rmse(predictions, test_data).compute()
        if verbose:
            print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

        weighted_mse[model_str].append(r)

        # Save models
        preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
        model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'
        
        predictions.to_netcdf(preds_nc_str)
        to_pickle(model_res, model_res_str)
    print('----------------------------------------------------------------------------')
    break

Running: dt_reg
0: Input variables = ['z']; output variables = ['z']
1: Input variables = ['t']; output variables = ['t']
2: Input variables = ['z', 't']; output variables = ['z', 't']
3: Input variables = ['t2m']; output variables = ['t2m']
4: Input variables = ['z', 't', 't2m']; output variables = ['z']
5: Input variables = ['z', 't', 't2m']; output variables = ['t']
6: Input variables = ['z', 't', 't2m']; output variables = ['t2m']
----------------------------------------------------------------------------


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
model_fun, model_str, extra_args =  [AdaBoostRegressor, 'adaboost', {'base_estimator': DecisionTreeRegressor(max_depth=5)}]

print('Running:', model_str)
for iter, (i, o) in enumerate(experiments):
    
    print(f'{iter}: Input variables = {i}; output variables = {o}')
    
    predictions, model_res, mse_train, mse_test = train_regression(model=model_fun,
                                                                    data=[data_train, data_test, nlat, nlon, data_std, data_mean],
                                                                    lead_time_h=lead_time, 
                                                                    input_vars=i, 
                                                                    output_vars=o,
                                                                    data_subsample=5,
                                                                    extra_args=extra_args,
                                                                    verbose=verbose)

    preds[model_str].append(predictions)
    models[model_str].append(model_res)

    # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
    r = compute_weighted_rmse(predictions, test_data).compute()
    if verbose:
        print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

    weighted_mse[model_str].append(r)

    # Save models
    preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
    model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'
    
    predictions.to_netcdf(preds_nc_str)
    to_pickle(model_res, model_res_str)
print('----------------------------------------------------------------------------')

In [10]:
# import xgboost as xgb # https://xgboost.readthedocs.io/en/latest/python/python_api.html

# preds_xgb_reg= []
# models_xgb_reg = []

# for iter, (i, o) in enumerate(experiments):
#     print(f'{iter}: Input variables = {i}; output variables = {o}')
#     pred, model = train_regression(xgb.XGBRFRegressor,
#                                    [data_train, data_test, nlat, nlon, data_std, data_mean],
#                                    lead_time, 
#                                    input_vars=i, 
#                                    output_vars=o,
#                                    extra_args={})
    
#     preds_xgb_reg.append(pred)
#     models_xgb_reg.append(model)

#     r = compute_weighted_rmse(pred, test_data).compute()
#     print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')